<a href="https://colab.research.google.com/github/TapasKumarDutta1/deep_neural_network_from_scratch_pneumonia_detection/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import cv2
import seaborn as sns
import numpy as np 
import gc
from numba import jit
import matplotlib.pyplot as plt
import math
np.random.seed(777)

import warnings
warnings.filterwarnings('ignore')

@jit
def pad1(a):
    b=np.ones((a.shape[0]+2,a.shape[1]+2))
    b[1:a.shape[0]+1,1:a.shape[1]+1]=a
    b[1:4,[0,-1]]=b[1:4,[1,-2]]
    b[[0,-1],:]=b[[1,-2],:]
    return b
@jit
def conv(inp,ker):
    inp=pad1(inp)
    v_k,h_k=ker.shape
    v_i,h_i=inp.shape
    out=np.zeros((1+v_i-v_k,1+h_i-h_k))
    for i in range(v_i):
        for j in range(h_i):
            if (i+v_k<=v_i) and (j+h_k<=h_i):
                out[i,j]=np.sum(inp[i:i+v_k,j:j+h_k]*ker)
    return out
filter=np.array([[0,-1,0], [-1,4,-1], [0,-1,0]])

from collections import Counter
@jit
def cnt(ground,count):
    tot=0
    for i in np.unique(ground):
        tot+=count[i]
    return tot
def otsu(img):
    count=Counter(img.ravel())
    tot=len(img.ravel())
    dk={}
    for val in np.unique(img):
        foreground=img[img>=val]
        background=img[img<val]
        f_count=cnt(foreground,count)
        b_count=cnt(background,count)
        dk[val]=(f_count*np.var(foreground)/tot)+(b_count*np.var(background)/tot)
    return min(dk, key=dk.get)        

#LAYERS
@jit
def sigmoid(x):
    return 1/(1+np.exp(x*-1))
@jit
def dot(a,b):
    return np.sum(a*b,axis=(-1,-2,-3))
@jit
def glorot(size1,in1,out1):
    dst=np.random.normal(0,1,size1)
    mx=np.max(dst)
    mn=np.min(dst)
    
    #wihin 0,1
    dst=(dst-mn)/(mx-mn)
    
    #within -1,1
    dst=(dst-0.5)*2
    
    
    scl=2.449489742783178/(np.sqrt(in1+out1))
    
    #within +-scl
    dst*=scl
    return dst
@jit
def maxpool(image,kernel):
    bs,l,w,c=image.shape
    out=np.zeros((bs,int(math.ceil(l/kernel)),int(math.ceil(w/kernel)),c))
    idxs=[]
    for i in range(int(math.ceil(l/kernel))):
        for j in range(int(math.ceil(w/kernel))):
            look=image[:,i*kernel:(i+1)*kernel,j*kernel:(j+1)*kernel,:]
            out[:,i,j,:]=np.max(look,axis=(1,2))
            idxs.append(np.where(np.in1d(look, out[:,i,j,:]))[0])
    return out,kernel,np.asarray(idxs),image
@jit
def convolution(image,kernel,out):
    bs,s1,s2,s3=image.shape
    b=np.zeros((bs,(s1-kernel)+1,(s2-kernel)+1,out))
    in1=len(image.ravel())/bs
    out1=len(b.ravel())/bs
    a=glorot((kernel,kernel,s3,out),in1,out1)
    for i in range(out):
        for j in range(b.shape[0]):
            for k in range(b.shape[1]):
                out=dot(a[:,:,:,i],image[:,j:j+kernel,k:k+kernel])
                b[:,j,k,i]=out
    return a,image,b
@jit
def flatten(image):
    bs,_,_,_=image.shape
    return image,image.reshape(bs,-1)
@jit
def dense(input1,out):
    bs,b=input1.shape
    in1=b
    out1=out
    weights=glorot((b,out),in1,out1)
    out=np.dot(input1,weights)
    return weights,input1,sigmoid(out)
@jit
def loss(true,pred):
    return -1*((true*np.log(pred))+((1-true)*(np.log(1-pred))))

#DERIVATIVE
@jit
def loss_back(true,pred):
    return pred-true

@jit
def rotate_180(array):
    M, N=array.shape
    out=np.zeros_like(array)
    for i in range(M):
        for j in range(N):
            out[i, N-1-j] = array[M-1-i, j]
    return out

@jit
def maxpool_derivative(derivative,kernel,idxs):
    bs,l,w,c=derivative.shape
    out=np.zeros((bs,int(l*kernel),int(w*kernel),c))
    en=0
    for i in range(l):
        for j in range(w):
            look=np.zeros((bs,kernel,kernel,c))
            place=idxs[en]
            look.ravel()[place]=1
            look*=derivative[:,i,j,:].reshape(bs,1,1,c)
            en+=1
            out[:,i*kernel:(i+1)*kernel,j*kernel:(j+1)*kernel,:]=look
    return out
@jit
def cng(grad,wts,x):
    bs,s1,s2,s3=x.shape
    grad1=np.zeros((bs,s1+1,s2+1,grad.shape[-1]))
    
    grad1[:,1:s1,1:s2,:]=grad
    
    out=np.zeros_like(x)
    p,q,r,s=wts.shape
    


    for l in range(s):
        for k in range(r):
            for i in range(s2):
                for j in range(s1):
                    out[:,i,j,k]+=dot(rotate_180(wts[:,:,k,l]),grad1[:,i:i+p,j:j+q,l])
    return out
@jit
def sigmoid_derivative(x):
    out=np.exp(-x)*(sigmoid(x)**2)
    return out
@jit
def conv(input,kernel):
    bs,s1,s2,z=input.shape
    _,q1,q2,x=kernel.shape
    b=np.zeros(((s1-q1)+1,(s2-q2)+1,z,x))
    for l in range(z):
      for y in range(x):
        for j in range(b.shape[0]):
            for k in range(b.shape[1]):
                out=dot(kernel[:,:,:,y],input[:,j:j+q1,k:k+q2,l])/input.shape[0]
                b[j,k,l,y]=out
    return b

#COMBINE

def calculate_grad(layers,true):
    for layer in layers:
        try:
            wts,inp,out,type,pos=layer['weight'],layer['input'],layer['output'],layer['type'],layer['position']
        except:
            try:
                out,shp,type,pos=layer['output'],layer['shape'],layer['type'],layer['position']
            except:
                out,type,pos,shp,index=layer['output'],layer['type'],layer['position'],layer['kernel'],layer['index']
        for i in range(0,pos):
            try:
                wts1,inp1,out1,type1,pos1=layers[i]['weight'],layers[i]['input'],layers[i]['output'],layers[i]['type'],layers[i]['position']
            except: 
                try:
                    out1,shp1,type1,pos1=layers[i]['output'],layers[i]['shape'],layers[i]['type'],layers[i]['position']
                except:
                    inp1,out1,type1,pos1,shp1,index1=layers[i]['input'],layers[i]['output'],layers[i]['type'],layers[i]['position'],layers[i]['kernel'],layers[i]['index']
            if i==0:
                grad=loss_back(true,out1)
                ot=sigmoid_derivative(out1)
                grad*=ot
                #32,1
            if type1=='out' and type!='out' and i!=pos-1:
                grad=np.dot(grad,wts1.T)
                #32,-1
            if type1=='max' and i!=pos-1:
                grad=maxpool_derivative(grad,shp1,index1)
            if type1=='flatten' and i!=pos-1:
                #32,237,237,6
                grad=grad.reshape(shp1)
            if type1=='conv'and i!=pos-1 :
                #
                grad=cng(grad,wts1,inp1)
        if type=='out' or type=='dense':
            grad1=np.dot(inp.T,grad)/true.shape[0]
            layer['grad']=grad1
        if type=='conv':
            grad1=conv(inp,grad)
            layer['grad']=grad1
            if layer['weight'].shape!=layer['grad'].shape:
                print(layer['weight'].shape,layer['grad'].shape,layer['type'])
    return layers



def propagate(layers,lr,itr):
    for layer in layers:
      if layer['type']!='flatten' and layer['type']!='max':
        try:
            layer['vdw']=0.9*layer['vdw']+0.1*layer['grad']
            layer['sdw']=0.99*layer['sdw']+0.01*layer['grad']**2
        except:
            layer['vdw']=0.1*layer['grad']
            layer['sdw']=0.01*layer['grad']**2
        layer['vcorr']=layer['vdw']/(0.9**itr)
        layer['scorr']=layer['sdw']/(0.99**itr)
        layer['weight']-=(lr*layer['vcorr']/((layer['scorr']**0.5)+1e-6))
    return layers






def update(layers):
    layers=layers[::-1]
    for en,layer in enumerate(layers):
        try:
            wts,inp,out,type,pos=layer['weight'],layer['input'],layer['output'],layer['type'],layer['position']
        except:
            try:
                out,shp,type,pos=layer['output'],layer['shape'],layer['type'],layer['position']
            except:
                out,type,pos,shp,index=layer['output'],layer['type'],layer['position'],layer['kernel'],layer['index']
        if type=='conv':
            bs,a,b,c=inp.shape
            bs,x,y,z=out.shape
            kernel=wts.shape[1]
            for i in range(z):
                for j in range(x):
                    for k in range(y):
                        out[:,j,k,i]=dot(wts[:,:,:,i],inp[:,j:j+kernel,k:k+kernel])
        if type=='out':
            out=np.dot(inp,wts)
            out=sigmoid(out)
        if pos!=1:
            layers[en+1]['input']=out
        layer['output']=out
    return layers[::-1]    
            

BATCH_SIZE = 32
IMG_HEIGHT = 240
IMG_WIDTH = 240
ALPHA = 2e-4

def get_data(data_dir):
    labels = ['PNEUMONIA', 'NORMAL']
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for en,img in enumerate(os.listdir(path)):
            if en==0:
                print(img)
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) 
                resized_arr = cv2.resize(img_arr, (IMG_WIDTH, IMG_HEIGHT))
                data.append([resized_arr, class_num])
            except Exception as e:
                pass

    return np.array(data)
train = get_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/train')



In [ ]:
from tqdm import tqdm

trn=[]
tar=[]
for i in tqdm(range(5216)):
    
    img=train[i][0]
    
    
    trn.append(img.reshape((1,240,240,1)))
    tar.append(train[i][1])

In [ ]:


trn=np.asarray(trn)/255
tar=np.asarray(tar)

trn=trn.reshape((5216,240,240,1))

from tqdm import tqdm
out=[]
true=[]
for e in tqdm(range(int(math.ceil(trn.shape[0]/32)))):
    img=[]
    target=[]
    for ii in range(e*32,(e+1)*32):
        if ii<trn.shape[0]:
            img.append(trn[ii].reshape(240,240,1))
            target.append(tar[ii])
    img=np.asarray(img)
    if e==0 or e==19:
            a=convolution(img,2,2)
            b=convolution(a[2],2,4)
            g=maxpool(b[2],2)
            f=flatten(g[0])
            o=dense(f[1],1)
            layers=[{'weight':np.load( '../input/makes-sense-mean-1/out06_weight.npy'),'input':o[1],'output':o[2],'type':'out','position':1},
                   {'shape':f[0].shape,'output':f[1],'type':'flatten','position':2},
                   {'output':g[0],'kernel':g[1],'index':g[2],'type':'max','position':3,'input':g[-1]},
                   {'weight':np.load( '../input/makes-sense-mean-1/conv36_weight.npy'),'input':b[1],'output':b[2],'type':'conv','position':4},
                   {'weight':np.load( '../input/makes-sense-mean-1/conv46_weight.npy'),'input':img,'output':a[2],'type':'conv','position':5}]
       
    layers[-1]['input']=img
    layers=update(layers)
    if e==0:
        out=layers[0]['output']
    else:
        out=np.concatenate([out,layers[0]['output']])
    true.append(target)



In [ ]:


def loss(true,pred):
    return np.mean(-1*((true*np.log(pred))+((1-true)*(np.log(1-pred)))))
loss(tar,out)



In [ ]:


np.save('predictions_train.npy',out)

np.save('train_targets.npy',tar)

def get_data(data_dir):
    labels = ['PNEUMONIA', 'NORMAL']
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for en,img in enumerate(os.listdir(path)):
            if en==0:
                print(img)
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) 
                resized_arr = cv2.resize(img_arr, (IMG_WIDTH, IMG_HEIGHT))
                data.append([resized_arr, class_num])
            except Exception as e:
                pass

    return np.array(data)
train = get_data('../input/chest-xray-pneumonia/chest_xray/chest_xray/test')



In [ ]:
trn=[]
tar=[]
for i in tqdm(range(624)):
    
    img=train[i][0]
    
    
    trn.append(img.reshape((1,240,240,1)))
    tar.append(train[i][1])

In [ ]:


trn=np.asarray(trn)/255
tar=np.asarray(tar)

from tqdm import tqdm
out=[]
true=[]
for e in tqdm(range(int(math.ceil(trn.shape[0]/32)))):
    img=[]
    target=[]
    for ii in range(e*32,(e+1)*32):
        if ii<trn.shape[0]:
            img.append(trn[ii].reshape(240,240,1))
            target.append(tar[ii])
    img=np.asarray(img)
    if e==0 or e==19:
            a=convolution(img,2,2)
            b=convolution(a[2],2,4)
            g=maxpool(b[2],2)
            f=flatten(g[0])
            o=dense(f[1],1)
            layers=[{'weight':np.load( '../input/makes-sense-mean-1/out06_weight.npy'),'input':o[1],'output':o[2],'type':'out','position':1},
                   {'shape':f[0].shape,'output':f[1],'type':'flatten','position':2},
                   {'output':g[0],'kernel':g[1],'index':g[2],'type':'max','position':3,'input':g[-1]},
                   {'weight':np.load( '../input/makes-sense-mean-1/conv36_weight.npy'),'input':b[1],'output':b[2],'type':'conv','position':4},
                   {'weight':np.load( '../input/makes-sense-mean-1/conv46_weight.npy'),'input':img,'output':a[2],'type':'conv','position':5}]
       
    layers[-1]['input']=img
    layers=update(layers)
    if e==0:
        out=layers[0]['output']
    else:
        out=np.concatenate([out,layers[0]['output']])
    true.append(target)



In [ ]:


def loss(true,pred):
    return np.mean(-1*((true*np.log(pred))+((1-true)*(np.log(1-pred)))))
loss(tar,out)



In [ ]:


np.save('predictions_test.npy',out)

np.save('test_targets.npy',tar)

import numpy as np
a=np.load('predictions_train.npy')
b=np.load('train_targets.npy')

from sklearn.metrics import accuracy_score
accuracy_score(np.round(a),b)



In [ ]:
a=np.load('predictions_test.npy')
b=np.load('test_targets.npy')

accuracy_score(np.round(a),b)



In [ ]:


from matplotlib import pyplot as plt
a=np.load('../input/makes-sense-mean-1/loss.npy')
plt.plot(a[:-1])

